# Project 16: Business Intelligence [BI]

In [751]:
# This block loads all required packages
from sqlalchemy import *
import pandas as pd

Since anaconda is used to manage environments, it's safer to store the creditentials used to connect to the database in environemnt variables.

In [752]:
# Establish a connection with the database using the environment variables
ssl_args = {'ssl_ca': "./files/ca-certificate.crt"}
sourceDb="classic_models_original"
destDb="classic_models"
srcEngine=create_engine(f'mysql+pymysql://{os.getenv("DB_USERNAME")}:{os.getenv("DB_PASSWORD")}@{os.getenv("DB_HOST")}:{os.getenv("DB_PORT")}/{sourceDb}', connect_args=ssl_args)
destEngine=create_engine(f'mysql+pymysql://{os.getenv("DB_USERNAME")}:{os.getenv("DB_PASSWORD")}@{os.getenv("DB_HOST")}:{os.getenv("DB_PORT")}/{destDb}', connect_args=ssl_args)

In [753]:
# Load payments from the original dataset
payments=pd.read_sql_table('Payments', srcEngine)
payments.head()

,customerNumber,checkNumber,paymentDate,amount
0,103,HQ336336,2004-10-19,5307.98
1,103,JM555205,2003-06-05,16560.30
2,103,OM314933,2004-12-18,2311.68
3,112,BO864823,2004-12-17,14449.61
4,112,HQ55022,2003-06-06,33847.62


In [754]:
# Load order details from the original dataset
orderDetails=pd.read_sql_table('OrderDetails', srcEngine)

# Drop the orderLineNumber column
orderDetails=orderDetails.drop(columns="orderLineNumber")
orderDetails.head()

,orderNumber,productCode,quantityOrdered,priceEach
0,10100,S18_1749,30,171.70
1,10100,S18_2248,50,67.80
2,10100,S18_4409,22,86.51
3,10100,S24_3969,49,34.47
4,10101,S18_2325,25,151.28


In [755]:
# Load products from the original dataset
products=pd.read_sql_table('Products', srcEngine)
products.head()

,productCode,productName,productLine,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP
0,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.70
1,S10_1949,1952 Alpine Renault 1300,Classic Cars,1:10,Classic Metal Creations,Turnable front wheels; steering function; deta...,7305,98.58,214.30
2,S10_2016,1996 Moto Guzzi 1100i,Motorcycles,1:10,Highway 66 Mini Classics,"Official Moto Guzzi logos and insignias, saddl...",6625,68.99,118.94
3,S10_4698,2003 Harley-Davidson Eagle Drag Bike,Motorcycles,1:10,Red Start Diecast,"Model features, official Harley Davidson logos...",5582,91.02,193.66
4,S10_4757,1972 Alfa Romeo GTA,Classic Cars,1:10,Motor City Art Classics,Features include: Turnable front wheels; steer...,3252,85.68,136.00


In [756]:
# Merge products and order details to insert relevant information into order details and orders tables
orderDetails=pd.merge(orderDetails, products[["productCode", "buyPrice"]], how='outer', on="productCode")
orderDetails=orderDetails.rename(columns={'buyPrice': 'costEach'})
orderDetails.head()

,orderNumber,productCode,quantityOrdered,priceEach,costEach
0,10100.0,S18_1749,30.0,171.7,86.7
1,10110.0,S18_1749,42.0,144.5,86.7
2,10124.0,S18_1749,21.0,136.0,86.7
3,10138.0,S18_1749,33.0,161.5,86.7
4,10149.0,S18_1749,34.0,158.1,86.7


In [757]:
# Create new orderTotals dataframe that contain the aggregate sale and cost price and profit for each ordernumber
orderTotals=orderDetails
orderTotals["totalSale"]=orderTotals["priceEach"]*orderTotals["quantityOrdered"]
orderTotals["totalCost"]=orderTotals["costEach"]*orderTotals["quantityOrdered"]
orderTotals["profit"]=orderTotals["totalSale"]-orderTotals["totalCost"]
orderTotals=orderTotals.groupby("orderNumber").agg("sum").reset_index()
orderTotals.head()


,orderNumber,quantityOrdered,priceEach,costEach,totalSale,totalCost,profit
0,10100.0,151.0,360.48,185.01,12133.25,6283.47,5849.78
1,10101.0,142.0,381.37,174.22,11432.34,5312.27,6120.07
2,10102.0,80.0,173.43,84.88,6864.05,3358.84,3505.21
3,10103.0,541.0,1642.25,937.33,54702.00,31391.84,23310.16
4,10104.0,443.0,1387.42,767.57,44621.96,24525.40,20096.56


In [758]:
# Load orders from the original dataset
orders=pd.read_sql_table('Orders', srcEngine)

# Merge orders with the previously created orderTotals dataframe
orders=pd.merge(orders, orderTotals[["orderNumber", "totalSale", "totalCost", "profit"]], how='outer', on="orderNumber")
orders.head()


,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber,totalSale,totalCost,profit
0,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,None,363,12133.25,6283.47,5849.78
1,10101,2003-01-09,2003-01-18,2003-01-11,Shipped,Check on availability.,128,11432.34,5312.27,6120.07
2,10102,2003-01-10,2003-01-18,2003-01-14,Shipped,None,181,6864.05,3358.84,3505.21
3,10103,2003-01-29,2003-02-07,2003-02-02,Shipped,None,121,54702.00,31391.84,23310.16
4,10104,2003-01-31,2003-02-09,2003-02-01,Shipped,None,141,44621.96,24525.40,20096.56


In [759]:
# Create new shipments table that contains all shipment related data
shipments=orders[["orderNumber", "orderDate", "requiredDate", "status", "shippedDate"]]
shipments["isLate"]=pd.to_datetime(shipments['requiredDate'], format='%Y-%m-%d') <= pd.to_datetime(shipments['shippedDate'], format='%Y-%m-%d')
shipments.head()

/var/folders/2t/12gb766d16vfd69jqf2t4_bh0000gn/T/ipykernel_4844/2183004676.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shipments["isLate"]=pd.to_datetime(shipments['requiredDate'], format='%Y-%m-%d') <= pd.to_datetime(shipments['shippedDate'], format='%Y-%m-%d')


,orderNumber,orderDate,requiredDate,status,shippedDate,isLate
0,10100,2003-01-06,2003-01-13,Shipped,2003-01-10,False
1,10101,2003-01-09,2003-01-18,Shipped,2003-01-11,False
2,10102,2003-01-10,2003-01-18,Shipped,2003-01-14,False
3,10103,2003-01-29,2003-02-07,Shipped,2003-02-02,False
4,10104,2003-01-31,2003-02-09,Shipped,2003-02-01,False


In [760]:
# Since the shipment related columns have moved to a seperate table, they can be dropped from the order table
orders=orders.drop(columns=["requiredDate", "status", "shippedDate"])
orders.head()

,orderNumber,orderDate,comments,customerNumber,totalSale,totalCost,profit
0,10100,2003-01-06,None,363,12133.25,6283.47,5849.78
1,10101,2003-01-09,Check on availability.,128,11432.34,5312.27,6120.07
2,10102,2003-01-10,None,181,6864.05,3358.84,3505.21
3,10103,2003-01-29,None,121,54702.00,31391.84,23310.16
4,10104,2003-01-31,None,141,44621.96,24525.40,20096.56


In [761]:
# Get total spent and profit for each customer
totalSpent=orders[["customerNumber", "totalSale", "profit"]].groupby("customerNumber").agg("sum")

# Add total amount paid
totalPaid=payments[["customerNumber", "amount"]].groupby("customerNumber").agg("sum")
totalSpent=pd.merge(totalSpent, totalPaid, how='outer', on="customerNumber")
totalSpent=totalSpent.rename(columns={'amount': 'totalPaid'})

totalSpent.head()

,totalSale,profit,totalPaid
customerNumber,,,
103,24179.96,11929.40,24179.96
112,82751.08,33882.82,82751.08
114,200995.41,90721.41,195364.73
119,180124.94,82427.12,136340.25
121,116599.19,53765.92,116599.19


In [762]:
# Load customers from the original dataset
customers=pd.read_sql_table('Customers', srcEngine)

# Merge the previously calculated totalSale and profit for each customer
customers=pd.merge(customers, totalSpent, how='outer', on="customerNumber")

# Also add column for current open credit of each customer
customers["credit"]=customers["totalSale"]-customers["totalPaid"]
customers.head()

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit,totalSale,profit,totalPaid,credit
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",None,Nantes,None,44000,France,1370.0,21000.0,24179.96,11929.40,24179.96,0.00
1,112,Signal Gift Stores,King,Sue,7025551838,8489 Strong St.,None,Las Vegas,NV,83030,USA,1166.0,71800.0,82751.08,33882.82,82751.08,0.00
2,114,"Australian Collectors, Co.",Ferguson,Peter,03 9520 4555,636 St Kilda Road,Level 3,Melbourne,Victoria,3004,Australia,1611.0,117300.0,200995.41,90721.41,195364.73,5630.68
3,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",None,Nantes,None,44000,France,1370.0,118200.0,180124.94,82427.12,136340.25,43784.69
4,121,Baane Mini Imports,Bergulfsen,Jonas,07-98 9555,Erling Skakkes gate 78,None,Stavern,None,4110,Norway,1504.0,81700.0,116599.19,53765.92,116599.19,0.00


In [763]:
# Load employees from the original dataset
employees=pd.read_sql_table('Employees', srcEngine)
employees.head()

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle
0,1002,Murphy,Diane,x5800,dmurphy@classicmodelcars.com,1,NaN,President
1,1056,Patterson,Mary,x4611,mpatterso@classicmodelcars.com,1,1002.0,VP Sales
2,1076,Firrelli,Jeff,x9273,jfirrelli@classicmodelcars.com,1,1002.0,VP Marketing
3,1088,Patterson,William,x4871,wpatterson@classicmodelcars.com,6,1056.0,"Sales Manager (JAPAN, APAC)"
4,1102,Bondur,Gerard,x5408,athompson@classicmodelcars.com,4,1056.0,Sale Manager (EMEA)


In [764]:
# Load offices from the original dataset
offices=pd.read_sql_table('Offices', srcEngine)
offices.head()

,officeCode,city,phone,addressLine1,addressLine2,state,country,postalCode,territory
0,1,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA
1,2,Boston,+1 215 837 0825,1550 Court Place,Suite 102,MA,USA,02107,NA
2,3,NYC,+1 212 555 3000,523 East 53rd Street,apt. 5A,NY,USA,10022,NA
3,4,Paris,+33 14 723 4404,43 Rue Jouffroy D'abbans,None,None,France,75017,EMEA
4,5,Tokyo,+81 33 224 5000,4-1 Kioicho,None,Chiyoda-Ku,Japan,102-8578,Japan


In [765]:
# Upload data to the destination database
destEngine.execute("SET SESSION sql_require_primary_key=0", index=False)
payments.to_sql('payments', destEngine, if_exists='replace', index=False)
orderDetails.to_sql('orderDetails', destEngine, if_exists='replace', index=False)
products.to_sql('products', destEngine, if_exists='replace', index=False)
shipments.to_sql('shipments', destEngine, if_exists='replace', index=False)
orders.to_sql('orders', destEngine, if_exists='replace', index=False)
customers.to_sql('customers', destEngine, if_exists='replace', index=False)
employees.to_sql('employees', destEngine, if_exists='replace', index=False)
offices.to_sql('offices', destEngine, if_exists='replace', index=False)

7